In [1]:

from Experiment import Experiment
import matplotlib.pyplot as plt
from Plotters import Plotters

%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name = ['manipulated_2023_08_10_100ms','manipulated_2022_02_03_dark','manipulated_2023_08_06_40ms',
                'manipulated_2023_08_09_60ms','manipulated_2023_08_07_5ms','manipulated_2023_08_07_10ms'
                ]#['2022_02_03','2022_03_10','2022_05_19','2022_03_03'] # 22_11_28

# exp_name = ['manipulated_2022_05_19_40ms']
ref_frame = 0

experiments = [Experiment(loadir,exp) for exp in exp_name]

min_max_v = [exp.min_max_point('CM_xy_dot') for exp in experiments]
min_max_pitch =[exp.min_max_point('pitch_y_frame') for exp in experiments]
zero_velocity = [exp.zero_velocity('CM_xy_dot') for exp in experiments]



In [19]:
import plotly.graph_objects as go
exp = experiments[0]

def plot_3d_traj_movie(mov):
    data,plot_cofnig = mov.calculation_for_3d_traj(color_prop = 'CM_xy_dot')
    ploter = Plotters(data,False)
    ploter.plot_3d_traj(data,plot_cofnig,mov.name,exp.experiment_name,color_prop = 'CM_xy_dot' )

plot_3d_traj_movie(exp.get_mov('mov22')) 




KeyError: 'mov22'

In [12]:

fig = go.Figure()
exp.plot_movies('time','CM_xy_dot', case = 'plot_mov', fig = fig, add_horizontal_line = 0,mov = False)
fig.show()


fig = go.Figure()
exp.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = fig, add_horizontal_line = 0,mov = False)
fig.show()

In [22]:
import numpy as np
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    CM_xy_dot = mov.get_prop('CM_xy_dot','body')
    CM_xy_dot = CM_xy_dot - np.mean(CM_xy_dot[0:mov.ref_frame])
    
    mov.data['body'] = np.hstack((mov.data['body'], CM_xy_dot[np.newaxis,:].T))
    mov.add_to_header(['CM_xy_dot_mean_zero'],'body')



c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [21]:
len(exp.exp_dict['mov1'].header['body'])

36

In [24]:

fig = go.Figure()
exp.plot_movies('time','CM_xy_dot_mean_zero', case = 'plot_mov', fig = fig, add_horizontal_line = 0,mov = False)
fig.show()


In [3]:

def plot_3d_traj_movie(mov):
    data,plot_cofnig = mov.calculation_for_3d_traj()
    ploter = Plotters(data,False)
    ploter.plot_3d_traj(data,plot_cofnig,mov.name,exp.experiment_name)

plot_3d_traj_movie(exp.get_mov('mov11')) 

# [plot_3d_traj_movie(exp.get_mov(mov_name)) for mov_name in exp.mov_names[73:74]]


In [14]:
import plotly.graph_objects as go
exp = experiments[2]

fig = go.Figure()
[exp.plot_movies('time','pitch_body', case = 'plot_mov', fig = fig, mov = False,add_horizontal_line=45,color = idx + 2) for idx,exp in enumerate(experiments[2:3])]
fig.show()


fig = go.Figure()
[exp.plot_movies('time','CM_xy_dot', case = 'plot_exp', fig = fig, add_horizontal_line = 0,mov = False,color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()


fig = go.Figure()
[exp.plot_movies('time','pitch_y_frame', case = 'plot_exp', fig = fig, add_horizontal_line = 0,mov = False,color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()



In [13]:
import numpy as np
import plotly.graph_objects as go


exp = experiments[2]
[exp.get_mov(mov_name).mean_props('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi') for mov_name in exp.mov_names]
[exp.get_mov(mov_name).mean_props('phi_rw','phi_lw','mean_wing','mean_mean_phi') for mov_name in exp.mov_names]

fig = go.Figure()
[exp.plot_movies('time','mean_front_phi', case = 'plot_mov',prop = 'mean_wing', fig = fig, mov = False,color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()

fig = go.Figure()
[exp.plot_movies('time','mean_mean_phi', case = 'plot_mov',prop = 'mean_wing', fig = fig, mov = False,color = idx + 2) for idx,exp in enumerate(experiments)]
fig.show()



KeyError: 'mean_front_phi'

In [5]:
import plotly.graph_objects as go

fig = go.Figure()    
[fig.add_trace(go.Histogram(x=np.hstack(min), name=exp.experiment_name, nbinsx=30)) for min,exp in zip(zero_velocity,experiments)]
fig.update_layout(title = 'time of minimum velocity')
fig.update_traces(opacity = 0.9)
fig.show()
